# Profundidad optica

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import integrate
from scipy.integrate import quad
from scipy import special
from mpmath import hyp1f1
import mpmath as mp
from scipy.integrate import dblquad
from multiprocessing import Pool
from functools import partial
import astropy.constants as c
import astropy.units as u
# import sympy as sp
import math

La profundidad Optica es
$$\tau = \int_0^{ D_S} \frac{\rho(x)}{m}\pi u_T^2 R_E(x)^2 dx$$
consideramos $x = D_L/D_S$, $u_T=1$, $D_S=8Kpc$

La cantidad $\frac{R_E(x)^2}{m} = \kappa \pi D_s^2 x(1-x)$

Por otro lado la densidad de esfera isoterma es
$$\rho(x) = \frac{R_{sol}^2+R_c^2}{R_c^2+r} $$

con $$ x = \sqrt{R_S^2 + (D_S x)^2 - 2 D_S Rs  x  \cos(l)\cos(b))}$$

In [19]:
rho_0 = 7.9e-3*c.M_sun/(u.pc)**3
pi    = np.pi
a     = 5*u.kpc 
R_0   = 8.5*u.kpc 
D_S   = np.array([1,5,10,55])*u.kpc

In [20]:
def integrand_OD_KG(x,angle,D_s):
    A     = (a**2+R_0**2)/D_s**2
    B     = R_0/D_s
    tau_0 = (4*pi*c.G*rho_0*(a**2+R_0**2))/(c.c**2) #4.61*10**-7
    return tau_0*x*(1-x)/(x**2+A-2*x*B*angle)

In [21]:
angle_lmc = np.cos(math.radians(-32.8))*np.cos(math.radians(281))
tau_lmc_0 = integrate.fixed_quad(lambda x: integrand_OD_KG(x,angle_lmc, D_S[3]), 0, 1,n=10)
tau_lmc   = tau_lmc_0[0]
print(tau_lmc.decompose())

5.104534452369212e-07


In [14]:
import numpy as np
import math
import astropy.units as u
import astropy.constants as c
from scipy.integrate import fixed_quad

def pos_sol(x):
    l = -32.8
    b = 281
    Ds = 55 * u.kpc
    angle = np.cos(math.radians(l)) * np.cos(math.radians(b))
    Rs = 8.5 * u.kpc
    r = np.sqrt(Rs**2 + (Ds * x)**2 - 2 * Rs * (x * Ds) * angle)
    return r

def rho_NFW(r):
    Rs = 21.5 * u.kpc
    rho0 = 4.88e+6 * c.M_sun / u.kpc**3
    x = r / Rs
    bot = x * (1 + x)**2
    rho = rho0 / bot
    return rho

def Einstein_angle_sq_m(x):
    Ds = 55 * u.kpc
    k = 4 * c.G / c.c**2
    return (k * x * (1 - x) * Ds**2)

def int_optical_depth(rho, x):
    '''
    Integrand for optical depth
    '''
    r = pos_sol(x)
    einstein_radius_sq_m = Einstein_angle_sq_m(x)
    optical_depth_integrand = (np.pi * rho(r) * einstein_radius_sq_m)
    return optical_depth_integrand.decompose().value  # Strip units for integration

# Perform the integration
optical_depth, _ = fixed_quad(lambda x: int_optical_depth(rho_NFW, x), 0, 1, n=10)
print("Integrated Optical Depth:", optical_depth)


Integrated Optical Depth: 3.943342232943051e-07
